In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

s_no = 20250056
url = f"https://statiz.sporki.com/schedule/?m=preview&s_no={s_no}"
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [2]:
# 점수 크롤링하는 함수
def crawl_score_values():
  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    div_nums = soup.select('div.num')

    if not div_nums:
      print("div.num을 찾을 수 없음")
      return []
    
    span_values = []
    
    for i, div_num in enumerate(div_nums):
            # 해당 div 안의 모든 span 태그 찾기
      spans = div_num.find_all('span')
            
      if spans:
        for j, span in enumerate(spans):
          text = span.get_text(strip=True)
          if text:  # 빈 텍스트가 아닌 경우만 추가
            span_values.append({
              'div_index': i + 1,
              'span_index': j + 1,
              'text': text
          })
          else:
            print(f"div.num[{i+1}] 안에 span 태그가 없습니다.")
        
    return span_values
  

  except requests.RequestException as e:
    print(f"웹페이지 요청 중 오류 발생: {e}")
    return []
  except Exception as e:
    print(f"크롤링 중 오류 발생: {e}")
    return []
  

# 날짜 크롤링 결과에서 숫자 값만 추출하는 함수
def extract_numeric_values(span_data):
    numeric_values = []
    
    for item in span_data:
        text = item['text']
        try:
            # 쉼표 제거 후 숫자로 변환 시도
            num_value = float(text.replace(',', ''))
            numeric_values.append({
                'div_index': item['div_index'],
                'span_index': item['span_index'],
                'original_text': text,
                'numeric_value': num_value
            })
        except ValueError:
            # 숫자가 아닌 경우 무시
            continue
    
    return numeric_values



# 숫자 값만을 가지고 데이터프레임을 생성하는 함수
def create_simple_numeric_dataframe(numeric_data):
    if not numeric_data:
        print("숫자 데이터가 없어서 DataFrame을 생성할 수 없습니다.")
        return pd.DataFrame()
    
    numeric_values_only = [item['numeric_value'] for item in numeric_data]
    
    df = pd.DataFrame({
        'value': numeric_values_only
    })
    
    return df

data = crawl_score_values()
numbers = extract_numeric_values(data)
score_df = pd.DataFrame({'총득점': [sum(item['numeric_value'] for item in numbers)]})



# 투수 정보를 크롤링하는 함수
def crawl_pitcher_values():
  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    # hometeam_player_name = soup.find_all('div', style="margin-top:.3rem;").get_text(strip=True)
    pitchers_name = [div.get_text(strip=True) for div in soup.find_all('div', style="margin-top:.3rem;")]

    return pitchers_name
  
  except Exception as e:
    print("오류 발생")
    return []
  
data2 = crawl_pitcher_values()

pitcher_df = pd.DataFrame(data2, columns=['선수명'])
pitcher_df = pitcher_df.transpose()
pitcher_df = pitcher_df.rename(columns={0: '홈 선발투수', 1: '원정 선발투수'})



# 경기 날짜를 크롤링하는 함수
def crawl_date_values():
  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    text = soup.select_one('div.txt').get_text(strip=True)
    date_data = re.search(r'\d{2}-\d{2}', text).group()

    return date_data
  
  except Exception as e:
    print("에러가 발생")
    
data3 = crawl_date_values()

date_df = pd.DataFrame([{'경기 날짜': data3}])



# url의 sno 데이터를 크롤링하는 함수
def crawl_sno_values():
  try:
    match = re.search(r's_no=(\d+)', url)
    s_no = match.group(1)

    return s_no
  
  except Exception as e:
    print("오류가 발생")

data4 = crawl_sno_values()
sno_df = pd.DataFrame([data4], columns=["s_no"])


merged_df = pd.concat([sno_df.reset_index(drop=True),
                       date_df.reset_index(drop=True),
                       score_df.reset_index(drop=True),
                       pitcher_df.reset_index(drop=True)
                       ], axis=1)

merged_df



,s_no,경기 날짜,총득점,홈 선발투수,원정 선발투수
0,20250056,04-04,10.0,양현종,송승기
